In [2]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:

df = pd.read_csv('D:\\DeepLearning\\Author-Attribution\\English_dataset\\Gungor_2018_VictorianAuthorAttribution_data-train.csv', encoding = "ISO-8859-1")
df['author'] = df['author'] -1
# df['text'] =df['text'].str.split()

In [4]:
df.head()

,text,author
0,ou have time to listen i will give you the ent...,0
1,wish for solitude he was twenty years of age a...,0
2,and the skirt blew in perfect freedom about th...,0
3,of san and the rows of shops opposite impresse...,0
4,an hour s walk was as tiresome as three in a s...,0


In [17]:
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()
# df['lemmatized'] = df['text'].apply(lambda x: [lemmatizer.lemmatize(word,pos='v') for word in x])

# df['text'] = [' '.join(map(str, l)) for l in df['lemmatized']]

In [4]:
df.head()

,text,author
0,ou have time to listen i will give you the ent...,0
1,wish for solitude he was twenty years of age a...,0
2,and the skirt blew in perfect freedom about th...,0
3,of san and the rows of shops opposite impresse...,0
4,an hour s walk was as tiresome as three in a s...,0


In [5]:
import re
df['text']= df['text'].apply(lambda x: x.lower())
df['text']= df['text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x))


In [6]:
df.head()

,text,author
0,ou have time to listen i will give you the ent...,0
1,wish for solitude he was twenty years of age a...,0
2,and the skirt blew in perfect freedom about th...,0
3,of san and the rows of shops opposite impresse...,0
4,an hour s walk was as tiresome as three in a s...,0


In [7]:
X = df['text']
Y = df['author']


In [22]:
# from nltk.corpus import stopwords
# print(stopwords.words('english'))
# vectorizer = CountVectorizer(stop_words=stopwords.words('english'),max_features=1000)
# X = vectorizer.fit_transform(X).toarray()


In [8]:
MAX_FEATURES = 1000
# 

In [9]:
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer(analyzer='word',stop_words=stopwords.words('english'),max_features=MAX_FEATURES)
X = vectorizer.fit_transform(X).toarray()

In [10]:
print(f"Shape of X: {X.shape}")

Shape of X: (53678, 1000)


In [12]:
features = vectorizer.get_feature_names_out()
df_post_processing = pd.DataFrame(X,columns=features).head()
df_post_processing.head()

,aa,able,according,account,acquaintance,across,act,action,added,advantage,...,ye,year,years,yellow,yes,yet,yon,york,young,youth
0,0.0,0.000000,0.0,0.000000,0.000000,0.055227,0.0,0.000000,0.000000,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.034050,0.000000,0.0,0.112247,0.000000
1,0.0,0.048354,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.04741,0.074913,0.0,0.0,0.061247,0.000000,0.0,0.134603,0.061496
2,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.00000,0.116900,0.0,0.0,0.031858,0.000000,0.0,0.140030,0.000000
3,0.0,0.000000,0.0,0.047266,0.058499,0.000000,0.0,0.057409,0.045014,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.027987,0.055117,0.0,0.030753,0.000000
4,0.0,0.044180,0.0,0.000000,0.058485,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.030746,0.000000


In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [14]:
X = np.array(X)
Y = np.array(Y).reshape(-1,1)
X.shape , Y.shape

((53678, 1000), (53678, 1))

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() 
Y = le.fit_transform(Y)

C:\Users\Shaima\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
import numpy as np  

x_train, x_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=11)

x_train.shape, y_train.shape

((42942, 1000), (42942,))

In [16]:
nb = MultinomialNB()
nb.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,nb.predict(x_test))}")

Accuracy of the model: 0.761456780923994


Make hypertuning for MultinomialNB()

In [21]:
from sklearn.model_selection import GridSearchCV
model=MultinomialNB()
# alpha is the laplace smoothing parameter
param={'alpha': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]}
clf=GridSearchCV(model,param,scoring='roc_auc',cv=5,return_train_score=True) 
clf.fit(x_train, y_train)
# Print the best hyperparameters
print("Best alpha:", clf.best_params_)


C:\Users\Shaima\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shaima\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Shaima\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Best alpha: {'alpha': 1e-05}


In [54]:
nb = MultinomialNB(alpha=0.1) # alpha to be lower is better
nb.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,nb.predict(x_test))}")

Accuracy of the model: 0.7697466467958272


In [55]:
nb = MultinomialNB(alpha=1e-05)
nb.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,nb.predict(x_test))}")

Accuracy of the model: 0.7704918032786885


making alpha=0.001

In [56]:
nb = MultinomialNB(alpha=0.001)
nb.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,nb.predict(x_test))}")

Accuracy of the model: 0.7705849478390462


In [32]:
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,lr.predict(x_test))}")

Accuracy of the model: 0.9141207153502235


C:\Users\Shaima\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


hypertuning for logistic regression

In [22]:
lr = LogisticRegression(solver='liblinear')
param_grid = {
    'C': [0.001, 0.01, 0.1,1],
    'penalty': ['l1', 'l2']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(lr, param_grid, scoring='roc_auc', cv=5)

# Fit the model to the training data
grid_search.fit(x_train, y_train)

# Print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

C:\Users\Shaima\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Shaima\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Shaima\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [62]:
lr = LogisticRegression(solver='saga')
lr.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,lr.predict(x_test))}")

Accuracy of the model: 0.9142138599105812


In [68]:
lr = LogisticRegression(l1_ratio=0.1,penalty='elasticnet',solver='saga',C=0.5) # adds more regularization by choosing smaller value of C (Inverse of regularization strength)
lr.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,lr.predict(x_test))}")

Accuracy of the model: 0.9027570789865872


Tuning KNN

In [31]:
from sklearn.model_selection import GridSearchCV

grid_params = { 'n_neighbors' : [5,9,11,15],
               'weights' : ['uniform','distance'],
               'metric' : ['euclidean','manhattan']}

In [32]:
gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=2, n_jobs = -1)


In [33]:
# fit the model on our train set
g_res = gs.fit(x_train, y_train)

Fitting 2 folds for each of 16 candidates, totalling 32 fits


In [34]:
g_res.best_score_

0.5183270457826836

In [35]:
g_res.best_params_

{'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}

In [20]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,knn.predict(x_test))}")

Accuracy of the model: 0.5690201192250373


In [38]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,knn.predict(x_test))}")

Accuracy of the model: 0.5451751117734724


In [22]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,knn.predict(x_test))}")

Accuracy of the model: 0.5227272727272727


In [23]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,knn.predict(x_test))}")

Accuracy of the model: 0.5012108792846498


In [39]:
knn = KNeighborsClassifier(n_neighbors=5,weights='distance',metric='euclidean')
knn.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,knn.predict(x_test))}")

Accuracy of the model: 0.5951005961251863


Tuning RandomForest classifier

In [19]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,rfc.predict(x_test))}")

Accuracy of the model: 0.6153129657228018


In [25]:
rfc = RandomForestClassifier(random_state=0,max_depth=70,n_estimators=400)
rfc.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,rfc.predict(x_test))}")

Accuracy of the model: 0.6331967213114754


In [28]:
rfc = RandomForestClassifier(random_state=0,max_depth=70,n_estimators=400,bootstrap=True,min_samples_leaf=4,min_samples_split=10,)
rfc.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,rfc.predict(x_test))}")

Accuracy of the model: 0.5939828614008942


In [17]:
rfc = RandomForestClassifier(random_state=0,max_depth=80,n_estimators=100,bootstrap=True,min_samples_leaf=5,min_samples_split=12,)
rfc.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,rfc.predict(x_test))}")

Accuracy of the model: 0.5779619970193741


In [27]:
rfc = RandomForestClassifier(random_state=0,criterion='log_loss')
rfc.fit(x_train,y_train)
print(f"Accuracy of the model: {accuracy_score(y_test,rfc.predict(x_test))}")

Accuracy of the model: 0.5560730253353204


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
scores =[]
for k in range(1, 100):
    rfc = RandomForestClassifier(n_estimators=k)
    rfc.fit(x_train, y_train)
    y_pred = rfc.predict(x_test)
    scores.append(accuracy_score(y_test, y_pred))

import matplotlib.pyplot as plt
%matplotlib inline

# plot the relationship between K and testing accuracy
# plt.plot(x_axis, y_axis)
plt.plot(range(1, 200), scores)
plt.xlabel('Value of n_estimators for Random Forest Classifier')
plt.ylabel('Testing Accuracy')

KeyboardInterrupt: 